In [ ]:
import pandas as pd

In [ ]:
sheet_name = 'captions_records_1000' # replace with your own sheet name
sheet_id = '1HOOTWxHksen9sOZTBCbRj66f10ULlgrnw1V8jVWurp8' # replace with your sheet's ID

In [ ]:
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [ ]:
data = pd.read_csv(url)
print(data.head())

  image_id  reference_caption  generated_caption
0  image_1                NaN                NaN


# BLEU(Bilingual Evaluation Understudy)

BLEU(Bilingual Evaluation Understudy) 점수는 기계 번역이나 자연어 생성 작업의 정량적 평가 방법 중 하나로, 생성된 텍스트가 참조 텍스트(reference text)와 얼마나 유사한지 측정하는 지표입니다. 주로 단어나 구문이 얼마나 일치하는지를 평가합니다.

In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

nltk.download('punkt')

# Smoothing function to avoid zero BLEU scores for short sentences
smoothie = SmoothingFunction().method4

# CSV 파일 불러오기
df = pd.read_csv('/content/drive/MyDrive/잡동사니/captions_records_1000.csv')

# BLEU 점수를 저장할 리스트
bleu_scores = []

# 전체 이미지 리스트
unique_images = df['image_id'].unique()

for image_id in unique_images:
    # 해당 이미지의 참조 캡셔닝과 생성된 캡셔닝을 가져오기
    image_data = df[df['image_id'] == image_id]
    reference_captions = image_data['reference_caption'].apply(lambda x: x.split()).tolist()  # 참조 캡셔닝을 단어 리스트로 변환
    generated_captions = image_data['generated_caption'].apply(lambda x: x.split()).tolist()  # 생성된 캡셔닝을 단어 리스트로 변환

    # 이미지에 대한 BLEU 점수 계산
    for generated_caption in generated_captions:
        score = sentence_bleu(reference_captions, generated_caption, smoothing_function=smoothie)
        bleu_scores.append(score)

# 전체 BLEU 점수 평균 계산
average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"전체 BLEU 점수 평균: {average_bleu * 100:.2f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


전체 BLEU 점수 평균: 13.41


BLEU 점수 해석

1.0 (100%): 완벽히 동일한 문장이 생성된 경우입니다. 생성된 텍스트가 참조 텍스트와 완전히 일치할 때 얻을 수 있는 최고 점수입니다.


0.0 (0%): 생성된 문장이 참조 문장과 전혀 일치하지 않는 경우입니다.

일반적인 점수 범위: BLEU 점수는 보통 0.0에서 1.0 사이의 값을 가지며(0%~100%), 30% 이상의 점수는 일반적으로 생성된 텍스트가 참조 텍스트와 꽤 유사하다는 의미입니다.

---
- 0.0 ~ 10.0: 매우 낮은 성능. 생성된 문장이 참조 문장과 거의 유사하지 않음.
- 10.0 ~ 30.0: 낮은 성능. 생성된 문장이 참조 문장과 다소 차이가 있음. 초보적인 모델에서 자주 볼 수 있음.
- 30.0 ~ 50.0: 중간 성능. 생성된 문장이 참조 문장과 어느 정도 유사하지만, 여전히 차이가 있음.
- 50.0 ~ 70.0: 좋은 성능. 생성된 문장이 참조 문장과 상당히 유사하며, 대부분의 중요한 정보가 일치함.
- 70.0 ~ 85.0: 매우 좋은 성능. 생성된 문장이 거의 참조 문장과 일치하며, 높은 수준의 성능을 나타냄.
- 85.0 이상: 완벽에 가까운 성능. 생성된 문장이 참조 문장과 거의 동일함.

# ROUGE (Recall-Oriented Understudy for Gisting Evaluation)

ROUGE (Recall-Oriented Understudy for Gisting Evaluation)는 자동 요약 및 자연어 처리에서 생성된 텍스트를 평가하기 위해 BLEU와 함께 자주 사용되는 평가 지표입니다.

ROUGE-1: 단일 단어(1-gram)의 일치율을 측정합니다.

ROUGE-2: 2-gram(단어의 연속된 두 단어)의 일치율을 측정합니다.

ROUGE-L: LCS(가장 긴 공통 서브시퀀스)를 기반으로 비교합니다.

평균 점수 계산: 각 캡셔닝에 대해 계산된 ROUGE 점수들을 모두 더한 후, 그 평균을 계산하여 최종 ROUGE-1, ROUGE-2, ROUGE-L 평균 점수를 출력합니다.

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=0ecc26dae23a1aa3188099f060b9d24c9af5635844eaa92e136cb5cebb0fcae4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

# ROUGE 평가를 위한 Scorer 초기화 (ROUGE-1, ROUGE-2, ROUGE-L을 계산)
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# ROUGE 점수를 저장할 리스트
rouge_scores = {
    'rouge1': [],
    'rouge2': [],
    'rougeL': []
}

# 전체 이미지 리스트
unique_images = df['image_id'].unique()

for image_id in unique_images:
    # 해당 이미지의 참조 캡셔닝과 생성된 캡셔닝을 가져오기
    image_data = df[df['image_id'] == image_id]
    reference_captions = image_data['reference_caption']
    generated_captions = image_data['generated_caption']

    # 각 생성된 캡셔닝에 대해 ROUGE 점수 계산
    for ref_cap, gen_cap in zip(reference_captions, generated_captions):
        scores = scorer.score(ref_cap, gen_cap)
        rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
        rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
        rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

# 각 ROUGE 스코어 평균 계산
average_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1'])
average_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2'])
average_rougeL = sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL'])

print(f"ROUGE-1 평균 점수: {average_rouge1 * 100:.2f}")
print(f"ROUGE-2 평균 점수: {average_rouge2 * 100:.2f}")
print(f"ROUGE-L 평균 점수: {average_rougeL * 100:.2f}")

ROUGE-1 평균 점수: 100.00
ROUGE-2 평균 점수: 0.00
ROUGE-L 평균 점수: 100.00


ROUGE 평가 결과 해석

ROUGE-1: 단어 일치율을 평가하는 지표로, 점수가 높을수록 생성된 캡셔닝이 참조 캡셔닝과 단어 수준에서 유사하다는 것을 의미합니다.

ROUGE-2: 2-gram 일치율을 평가하는 지표로, 점수가 높을수록 두 단어의 연속된 묶음이 참조 캡셔닝과 유사하게 생성되었음을 의미합니다.

ROUGE-L: 가장 긴 공통 서브시퀀스를 평가하는 지표로, 점수가 높을수록 전체 문장에서 긴 구문 구조가 참조 캡셔닝과 유사하다는 것을 의미합니다.

---
- 0.0 ~ 10.0: 매우 낮은 성능. 생성된 텍스트가 참조 텍스트와 거의 일치하지 않음.

- 10.0 ~ 30.0: 낮은 성능. 생성된 텍스트가 참조 텍스트와 다소 차이가 있음.

- 30.0 ~ 50.0: 중간 성능. 생성된 텍스트가 참조 텍스트와 어느 정도 유사하지만, 여전히 차이가 있음.

- 50.0 ~ 70.0: 좋은 성능. 생성된 텍스트가 참조 텍스트와 상당히 유사하며, 중요한 정보가 일치함.

- 70.0 ~ 85.0: 매우 좋은 성능. 생성된 텍스트가 참조 텍스트와 거의 일치함.

- 85.0 이상: 거의 완벽한 성능.

# METEOR(Metric for Evaluation of Translation with Explicit ORdering)

METEOR(Metric for Evaluation of Translation with Explicit ORdering)는 BLEU와 ROUGE와 함께 자주 사용되는 자연어 처리(NLP) 평가 지표 중 하나입니다. METEOR는 단어의 어근, 동의어, 그리고 단어 순서까지 고려하여 평가를 진행하므로, BLEU나 ROUGE보다 더 의미 중심적인 평가를 할 수 있습니다.

In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
import nltk
from nltk.translate.meteor_score import meteor_score

# NLTK에서 WordNet 리소스를 다운로드
nltk.download('wordnet')

# CSV 파일 불러오기
df = pd.read_csv('/content/drive/MyDrive/잡동사니/captions_records_1000.csv')

# METEOR 점수를 저장할 리스트
meteor_scores = []

# 전체 이미지 리스트
unique_images = df['image_id'].unique()

for image_id in unique_images:
    # 해당 이미지의 참조 캡셔닝과 생성된 캡셔닝을 가져오기
    image_data = df[df['image_id'] == image_id]
    reference_captions = image_data['reference_caption'].tolist()  # 참조 캡셔닝 리스트
    generated_captions = image_data['generated_caption'].tolist()  # 생성된 캡셔닝 리스트

    # 각 생성된 캡셔닝에 대해 METEOR 점수 계산
    for ref_cap, gen_cap in zip(reference_captions, generated_captions):
        # 참조 캡셔닝과 생성된 캡셔닝 모두를 단어 단위로 토큰화
        ref_cap_tokens = ref_cap.split()  # 참조 캡셔닝을 단어로 분리
        gen_cap_tokens = gen_cap.split()  # 생성된 캡셔닝을 단어로 분리

        # METEOR 점수를 계산하여 리스트에 추가
        score = meteor_score([ref_cap_tokens], gen_cap_tokens)  # 토큰화된 리스트로 전달
        meteor_scores.append(score)

# 전체 METEOR 점수 평균 계산
average_meteor = sum(meteor_scores) / len(meteor_scores)
print(f"전체 METEOR 점수 평균: {average_meteor * 100:.2f}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


전체 METEOR 점수 평균: 51.11


- 0.0 ~ 10.0: 매우 낮은 성능. 생성된 문장이 참조 문장과 거의 유사하지 않음.
- 10.0 ~ 30.0: 낮은 성능. 생성된 문장이 참조 문장과 다소 차이가 있음.
- 30.0 ~ 50.0: 중간 성능. 생성된 문장이 참조 문장과 어느 정도 유사하지만, 여전히 차이가 있음.
- 50.0 ~ 70.0: 좋은 성능. 생성된 문장이 참조 문장과 상당히 유사하며, 중요한 정보가 일치함.
- 70.0 이상: 매우 좋은 성능. 생성된 문장이 참조 문장과 거의 일치하며, 높은 수준의 의미적 유사성을 나타냄.